In [1]:
# Bike Sharing Demand Kaggle project

# W207 Final Project

# Chris Murray, Rahul Ragunathan,  Mahadevan Rajagopalan

#
# Import the needed libraries.
import csv
import re
import datetime
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# setup the training and test data
def read_data(filename, is_train, use_casual):

    with open(filename, 'rb') as csvfile:
       # skip the header
       header_line = next(csvfile)
       csvreader = csv.reader(csvfile, delimiter=',')
       # Initialize some empty lists
       time       = []
       season     = []
       holiday    = []
       workingday = []
       weather    = []
       temp       = []
       atemp      = []
       humidity   = []
       windspeed  = []
       # There are three additional fields with training data
       if is_train:
         casual     = []
         registered = []
         count      = []
       # Each row is a collection of strings
       for row in csvreader:
          time.append(row[0])
          season.append((int)(row[1]))
          holiday.append((int)(row[2]))
          workingday.append((int)(row[3]))
          weather.append((int)(row[4]))
        
          # Convert the float values to int values
          temp.append((int)(float(row[5])))
          atemp.append((int)(float(row[6])))
          humidity.append(int(float(row[7])))
          windspeed.append(int(float(row[8])))
          if is_train:
            casual.append(int(row[9]))
            registered.append(int(row[10]))
            count.append(int(row[11]))
            
       # The time is in year month day hour min secs  format.
       # Get the month and the day of the week.
       day_of_week =[]
       month =[]
       for str in time:
          day_of_week.append(datetime.datetime.weekday(datetime.datetime.strptime(str, "%Y-%m-%d %H:%M:%S")))
          month.append(datetime.datetime.weekday(datetime.datetime.strptime(str, "%Y-%m-%d %H:%M:%S")))
        
       # We just need the time of the day. This will be converted into the number of seconds
       # since the start of the day.
       splittime = []
       for str in time:
         splittime.append( str.split(" ")[1])
        
       # convert to number of seconds
       secondstime=[]
       for str in splittime:
         h,m,s = re.split(':',str)
         secondstime.append("%d" % (int)(datetime.timedelta(hours=int(h),minutes=int(m),seconds=int(s)).total_seconds()))
       
       num_features = 9
       
       data = np.zeros((len(season), num_features), dtype=np.int)
       # temp and atemp are probably correlated. Eliminating temp which has lower variance.
       # After adding the day of the week, removing holiday
       # copy the the feature values to the data array
       for i in range(data.shape[0]):
          data[i][0] = month[i]
          data[i][1] = day_of_week[i]
          data[i][2] = secondstime[i]
          data[i][3] = workingday[i]
          data[i][4] = weather[i]
          data[i][5] = atemp[i]
          data[i][6] = humidity[i]
          data[i][7] = windspeed[i]
          data[i][8] = season[i]
          
            
       if is_train:
          if use_casual:
            return data, casual
          else:
            return data, registered
       else:
          return data, time
            
# This function writes a row to a csv file.    
def write_csv(filename, row):
    b = open(filename, 'w')
    writer = csv.writer(b)
    
    writer.writerows(row)
    #for i in range(len(preds)+1):
    #  writer.writerow(row[i])
      
    b.close()
    return

# Read the training data   
train_data_casual, train_labels_casual = read_data('train.csv', True, True)
train_data_reg, train_labels_reg = read_data('train.csv', True, False)

# Read the test data 
test_data, time = read_data('test.csv', False, False)

# Instantiate the classifier

clf =   DecisionTreeClassifier(random_state=0)

# Fit the training data
clf.fit(train_data_casual, train_labels_casual)

# Make predictions
preds= clf.predict(test_data)

clf2 = DecisionTreeClassifier(random_state=0)
clf2.fit(train_data_reg, train_labels_reg)
preds2= clf2.predict(test_data)

l= ["datetime", "count"]
for i in range (len(preds)):
    l.append(time[i]) 
    sum = preds[i] +preds2[i]
    l.append("%d" %sum)
            
pred_output = np.array(l).reshape((len(preds)+1, 2))

print pred_output
# Write the predictions to a csv file
write_csv('DecisionTree11.csv', pred_output)


      

      
    
    

        
    


    

    
    
                          
        
                          
    
        
    
    
    
        
        
    
    

[['datetime' 'count']
 ['2011-01-20 00:00:00' '12']
 ['2011-01-20 01:00:00' '6']
 ..., 
 ['2012-12-31 21:00:00' '51']
 ['2012-12-31 22:00:00' '42']
 ['2012-12-31 23:00:00' '41']]
